# Examples for functions from utils.general module.

Some functions have optional parameter mode (=MODE_NP by default). If mode is MODE_NP (=0) then numpy format is used for all calculations, if mode is MODE_TT (=1) then tensor train (TT) format is used, if mode is MODE_SP (=2) then scipy sparse format is used.

In [1]:
import numpy as np
from scipy.sparse import diags as sp_diag

import tt

from qttpdesolver.utils.general import MODE_NP, MODE_TT, MODE_SP, vdiag
from qttpdesolver.utils.block_and_space import kronm, mblock, vblock, space_kron, kron_out, sum_out

## Vectors and matrices for examples

The vectors and matrices are constructed here in numpy format and then are transformed to [quantized] tensor train format (the last ones are named according to mask *_tt).

In [2]:
d = 2
n = 2**d
tau = 1.E-8
x = np.arange(n)+1.
y =-2.*x
z = x*y

x_tt = tt.tensor(x.reshape([2]*d, order='F'), tau)
y_tt = tt.tensor(y.reshape([2]*d, order='F'), tau)
z_tt = tt.tensor(z.reshape([2]*d, order='F'), tau)

A_tt = tt.Toeplitz(x_tt, d, kind='U')
A = A_tt.full()

B_tt = tt.Toeplitz(y_tt, d, kind='L')
B = B_tt.full()

C_tt = tt.Toeplitz(z_tt, d, kind='L')
C = C_tt.full()

I = np.eye(n)
I_sp = sp_diag([np.ones(n)], [0], format='csr')

## Examples

In [3]:
help(mblock)
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])
c = np.array([[-1, -2], [-3, -4]])
dd = np.array([[-5, -6], [-7, -8]])
print mblock([[tt.matrix(a), tt.matrix(b)], [tt.matrix(c), tt.matrix(dd)]]).full()
print mblock([[a, b], [c, dd]])
print mblock([[a, b], [c, None]])
print mblock([[a, b], [2, dd]])
print mblock([[a, None], [c, dd]])
print mblock([[0, b], [c, dd]])
print mblock([[a, b], [0, None]])
print mblock([[0, None], [c, dd]])
print mblock([[0, None], [c, 0]])
print mblock([[0, 0], [0, 0]])

Help on function mblock in module qttpdesolver.utils.block_and_space:

mblock(lst, tau=None)
    Construct rectangular block matrix from list (or list of lists) lst,
    that contains matrices of equal shape and maybe some (not all!)
    int/float or None entries. For None and int/float entries, corresponding
    elements in block matrix will be filled by zero-marices.
            Input:
    lst     - a given list of matrices in QTT-format
              type: list or list of lists of np.ndarray/tt.matrix/scipy csr
    tau     - is an accuracy for rounding operation
              (if it's None, then rounding will not be performed)
              type: float or None (default: None)
            Output:
    res     - block matrix
              type: is the same as type of matrices in lst

[[ 1.  2.  5.  6.]
 [ 3.  4.  7.  8.]
 [-1. -2. -5. -6.]
 [-3. -4. -7. -8.]]
[[ 1.  2.  5.  6.]
 [ 3.  4.  7.  8.]
 [-1. -2. -5. -6.]
 [-3. -4. -7. -8.]]
[[ 1.  2.  5.  6.]
 [ 3.  4.  7.  8.]
 [-1. -2.  0.

In [5]:
help(vblock)
print np.array(vblock([x, y, z]), dtype=np.int16)
print np.array(vblock([x, y, None, z]), dtype=np.int16)
print np.array(vblock([x_tt, y_tt, z_tt], tau).full().flatten('F'), dtype=np.int16)

Help on function vblock in module qttpdesolver.utils.block_and_space:

vblock(lst, tau=None)
    Concatenate vectors from a list lst. All vectors must have the same length.
    List may contain some (not all!) int/float or None entries, that will be
    filled by zero-vectors.
            Input:
    lst     - a given list of vectors
              type: list of np.ndarray(1D) or tt.vector
    tau     - is an accuracy for rounding operation
              (if it's None, then rounding will not be performed)
              type: float or None (default: None)
            Output:
    res     - concatenation
              type: is the same as type of vectors in lst

[  1   2   3   4  -2  -4  -6  -8  -2  -8 -18 -32]
[  1   2   3   4  -2  -4  -6  -8   0   0   0   0  -2  -8 -18 -32]
[  1   2   3   4  -2  -4  -6  -8  -2  -8 -18 -32]


In [6]:
help(kronm)
print kronm([x, y, z], tau)
print kronm([x_tt, y_tt, z_tt], tau).full().flatten('F')
print kronm([A, B])
print kronm([A_tt, B_tt], tau).full()
print vdiag(kronm([I_sp, I_sp]))

Help on function kronm in module qttpdesolver.utils.block_and_space:

kronm(lst, tau=None)
    Compute Kronecker products for given list of matrices or vectors.
    (If tt-format is used then list is reversed.)

[    4.    16.    36.    64.     8.    32.    72.   128.    12.    48.
   108.   192.    16.    64.   144.   256.     8.    32.    72.   128.
    16.    64.   144.   256.    24.    96.   216.   384.    32.   128.
   288.   512.    12.    48.   108.   192.    24.    96.   216.   384.
    36.   144.   324.   576.    48.   192.   432.   768.    16.    64.
   144.   256.    32.   128.   288.   512.    48.   192.   432.   768.
    64.   256.   576.  1024.]
[    4.    16.    36.    64.     8.    32.    72.   128.    12.    48.
   108.   192.    16.    64.   144.   256.     8.    32.    72.   128.
    16.    64.   144.   256.    24.    96.   216.   384.    32.   128.
   288.   512.    12.    48.   108.   192.    24.    96.   216.   384.
    36.   144.   324.   576.    48.   192.   432

In [7]:
help(space_kron)
print x
print space_kron(x, 0, d, n, dim=1)
print space_kron(x, 0, d, n, dim=2)
print space_kron(x, 1, d, n, dim=2)
print space_kron(x_tt, 1, d, n, 2, tau).full().flatten('F')
print space_kron(x, 1, d, n, dim=3)

X = np.ones((2, 2))
print space_kron(X, 0, d, n, dim=2)
print space_kron(X, 1, d, n, dim=2)

Help on function space_kron in module qttpdesolver.utils.block_and_space:

space_kron(x, axis, d, n, dim, tau=None)
    Prepare multidimensional (dim) version of the 1d operator x (in a matrix
    form) or 1d vector by corresponding Kronecker products with edentity matrix
    or edentity vector respectively for given axis.

[ 1.  2.  3.  4.]
[ 1.  2.  3.  4.]
[ 1.  2.  3.  4.  1.  2.  3.  4.  1.  2.  3.  4.  1.  2.  3.  4.]
[ 1.  1.  1.  1.  2.  2.  2.  2.  3.  3.  3.  3.  4.  4.  4.  4.]
[ 1.  1.  1.  1.  2.  2.  2.  2.  3.  3.  3.  3.  4.  4.  4.  4.]
[ 1.  1.  1.  1.  2.  2.  2.  2.  3.  3.  3.  3.  4.  4.  4.  4.  1.  1.
  1.  1.  2.  2.  2.  2.  3.  3.  3.  3.  4.  4.  4.  4.  1.  1.  1.  1.
  2.  2.  2.  2.  3.  3.  3.  3.  4.  4.  4.  4.  1.  1.  1.  1.  2.  2.
  2.  2.  3.  3.  3.  3.  4.  4.  4.  4.]
[[ 1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0

In [2]:
help(sum_out)
d = 1
n = 2**d
h = 1./n
x = np.arange(n)+1
print '1D case: ', x
print sum_out(x, 0, d, n, dim=1)
print 'tt of previous: '
print sum_out(tt.vector(x.reshape([2]*(d), order='F')), 0, d, n, dim=1, tau=1.E-8).full().flatten('F')
x = np.arange(n*n)+1
print '2D case: ', x
print sum_out(x, 0, d, n, dim=2)
print sum_out(x, 1, d, n, dim=2)
print 'tt of previous: ',
print sum_out(tt.vector(x.reshape([2]*(2*d), order='F')), 1, d, n, dim=2, tau=1.E-8).full().flatten('F')
x = np.arange(n*n*n)+1
print '3D case: ', x
print sum_out(x, 0, d, n, dim=3)
print sum_out(x, 1, d, n, dim=3)
print sum_out(x, 2, d, n, dim=3)
print 'tt of previous: ', 
print sum_out(tt.vector(x.reshape([2]*(3*d), order='F')), 2, d, n, dim=3, tau=1.E-8).full().flatten('F')

Help on function sum_out in module qttpdesolver.utils.block_and_space:

sum_out(x, axis, d, n, dim, tau=None)
    Fast calculation of the special product for dim=2,3 (for =1 x is returned) 
    S[axis] diag(x) S[axis].T, where x is a vector of size (n**dim) and
    for dim==2: S[0] = I \kron e.T
                S[1] = e.T \kron I
    for dim==3: S[0] = I \kron I \kron e.T
                S[1] = I \kron e.T \kron I
                S[2] = e.T \kron I \kron I
    where I is an (n, n) eye matrix and e is an (n, 1) vector of ones.
    The result is a diagonal matrix, and the function returns it's diagonal,
    which is a vector of size (n**(dim-1)).

1D case:  [1 2]
[1 2]
tt of previous: 
[ 1.  2.]
2D case:  [1 2 3 4]
[3 7]
[4 6]
tt of previous:  [ 4.  6.]
3D case:  [1 2 3 4 5 6 7 8]
[ 3  7 11 15]
[ 4  6 12 14]
[ 6  8 10 12]
tt of previous:  [  6.   8.  10.  12.]


In [5]:
help(kron_out)
d = 1
n = 2**d
h = 1./n
x = np.arange(n)+1
print '1D case: ', x
print kron_out(x, 0, d, n, dim=1)
print 'tt of previous: '
print kron_out(tt.vector(x.reshape([2]*(d), order='F')), 0, d, n, dim=1, tau=1.E-8).full()
x = np.arange(n)+1
print '2D case: ', x
print kron_out(x, 0, d, n, dim=2)
print kron_out(x, 1, d, n, dim=2)
print 'tt of previous: '
print kron_out(tt.vector(x.reshape([2]*(d), order='F')), 1, d, n, dim=2, tau=1.E-8).full()
x = np.arange(n*n)+1
print '3D case: ', x
print kron_out(x, 0, d, n, dim=3)
print kron_out(x, 1, d, n, dim=3)
print kron_out(x, 2, d, n, dim=3)
print 'tt of previous: '
print kron_out(tt.vector(x.reshape([2]*(2*d), order='F')), 2, d, n, dim=3, tau=1.E-8).full()

Help on function kron_out in module qttpdesolver.utils.block_and_space:

kron_out(x, axis, d, n, dim, tau=None)
    Fast calculation of the special product for dim=2,3 (for =1 A is returned) 
    S[axis].T diag(x) S[axis], where x is a vector of size (n**(dim-1)) and
    for dim==2: S[0] = I \kron e.T
                S[1] = e.T \kron I
    for dim==3: S[0] = I \kron I \kron e.T
                S[1] = I \kron e.T \kron I
                S[2] = e.T \kron I \kron I
    where I is an (n, n) eye matrix and e is an (n) vector of ones.
    The function return resulting matrix of shape (n**dim, n**dim).

1D case:  [1 2]
[1 2]
tt of previous: 
[ 1.  2.]
2D case:  [1 2]
[[ 1.  1.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  2.  2.]]
[[ 1.  0.  1.  0.]
 [ 0.  2.  0.  2.]
 [ 1.  0.  1.  0.]
 [ 0.  2.  0.  2.]]
tt of previous: 
[[ 1.  0.  1.  0.]
 [ 0.  2.  0.  2.]
 [ 1.  0.  1.  0.]
 [ 0.  2.  0.  2.]]
3D case:  [1 2 3 4]
[[ 1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0